# LSTM

LSTMs are RNNs designed to handle sequential data and are easily adaptable for sentiment analysis. They're designed to maintain memory cells and gates to capture long-range patterns that traditional RNNs fail to do so per the vanishing gradient problem.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv('sentiment140_cleaned.csv')
texts = df['clean_text'].astype(str)
labels = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size = 0.2, random_state = 734)

Let's start off with a tokenizer and look to build a vocabulary. For our embeddings, we'll load in pre-trained embeddings (GloVe). We will also look to do compute heavy tasks (training) on GPU.

In [2]:
import torch
from torchtext.vocab import GloVe, build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence
import torch

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# Pre-trained embeddings
glove = GloVe(name = '6B', dim = 100)

# Tokenizer
tokenizer = get_tokenizer("basic_english")
tokenized_texts = [tokenizer(text) for text in X_train]

# Vocab
def yield_tokens(tokenized_texts):
    for tokens in tokenized_texts:
        yield tokens

vocab = build_vocab_from_iterator(
    yield_tokens(tokenized_texts),
    specials=["<unk>", "<pad>"]
)
vocab.set_default_index(vocab["<unk>"])

# Embedding matrix
embedding_matrix = torch.zeros(len(vocab), 100)
for i, token in enumerate(vocab.get_itos()):
    if token in glove.stoi:
        embedding_matrix[i] = glove[token]
    else:
        embedding_matrix[i] = torch.randn(100)  # random for out of vocab tokens

Now we have our embeddings/vocab setup so we can transform our text into tensors and define a dataset/dataloader to load our data into the model for training.

In [4]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader

def text_pipeline(text, tokenizer, vocab):
    return torch.tensor([vocab[token] for token in tokenizer(text)], dtype=torch.long)

# Convert train/test into indexed tensors
X_train_indices = [text_pipeline(text, tokenizer, vocab) for text in X_train]
X_test_indices  = [text_pipeline(text, tokenizer, vocab) for text in X_test]

# Pad our sequences
X_train_padded = pad_sequence(X_train_indices, batch_first=True, padding_value=vocab["<pad>"])
X_test_padded  = pad_sequence(X_test_indices, batch_first=True, padding_value=vocab["<pad>"])

# Convert labels into tensors
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long)

# Dataset
train_dataset = TensorDataset(X_train_padded, y_train_tensor)
test_dataset  = TensorDataset(X_test_padded, y_test_tensor)

# Data Loader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64)


Now we're ready to define the LSTM classifier!

In [19]:
import torch.nn as nn

class LSTMSentiment(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, embedding_matrix, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.embedding.weight.data.copy_(embedding_matrix)
        self.embedding.weight.requires_grad = False

        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        embedded = self.embedding(x)
        _, (hidden, _) = self.lstm(embedded)
        hidden = hidden[-1]
        out = self.fc(hidden)
        return out


In [20]:
import torch.optim as optim

model = LSTMSentiment(
    vocab_size = len(vocab),
    embedding_dim = 100,
    hidden_dim = 128,
    output_dim = 2,
    embedding_matrix = embedding_matrix,
    pad_idx = vocab["<pad>"]
)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total_loss, correct, total = 0, 0, 0
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    acc = correct / total
    print(f"Epoch {epoch + 1}/{5}, Loss: {total_loss/len(train_loader):.4f}, Acc: {acc:.4f}")

Epoch 1/5, Loss: 0.6960, Acc: 0.4963
Epoch 2/5, Loss: 0.6940, Acc: 0.4973
Epoch 3/5, Loss: 0.6751, Acc: 0.5856
Epoch 4/5, Loss: 0.6099, Acc: 0.6719
Epoch 5/5, Loss: 0.5718, Acc: 0.7031
Epoch 6/5, Loss: 0.5450, Acc: 0.7239
Epoch 7/5, Loss: 0.5255, Acc: 0.7392
Epoch 8/5, Loss: 0.4988, Acc: 0.7554
Epoch 9/5, Loss: 0.4652, Acc: 0.7823
Epoch 10/5, Loss: 0.4248, Acc: 0.8059


In [21]:
model.eval()
correct, total = 0, 0

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        preds = outputs.argmax(1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Test Accuracy: {correct/total:.4f}")

Test Accuracy: 0.6965
